In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import plotly as py
import cufflinks as cf
cf.go_offline()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math
from IPython.display import display, HTML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate

In [2]:
cannabis = pd.read_csv('../data/cannabis.csv')
cannabis = cannabis[cannabis["Rating"] >= 4.5]

cannabis.to_csv(r'../data/cannabis_effects.csv')
cannabis.shape

(1151, 6)

In [3]:
cannabis.head()
data = pd.read_csv('../data/cannabis_effects.csv')
data.head()
data_effect = pd.DataFrame(data.Effects.str.split(',',4).tolist(),
             columns = ['Effect_1','Effect_2','Effect_3','Effect_4','Effect_5'])

data = pd.concat([data, data_effect], axis=1)
del data["Effects"]
del data["Flavor"]
del data["Unnamed: 0"]
data.head()

,Strain,Type,Rating,Effects,Flavor,Description
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
8,3D-Cbd,sativa,4.6,"Uplifted,Focused,Happy,Talkative,Relaxed","Earthy,Woody,Flowery",3D CBD from Snoop Dogg’s branded line of canna...
10,3Rd-Coast-Panama-Chunk,hybrid,4.7,"Energetic,Happy,Talkative,Uplifted","Pine,Diesel",3rd Coast Panama Chunk by Bedford Grow is a cr...
11,501St-Og,hybrid,4.5,"Relaxed,Euphoric,Sleepy,Happy,Talkative","Earthy,Pine,Skunk","501st OG, bred by Rare Dankness, is an indica-..."


,Unnamed: 0,Strain,Type,Rating,Effects,Flavor,Description
0,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
1,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
2,8,3D-Cbd,sativa,4.6,"Uplifted,Focused,Happy,Talkative,Relaxed","Earthy,Woody,Flowery",3D CBD from Snoop Dogg’s branded line of canna...
3,10,3Rd-Coast-Panama-Chunk,hybrid,4.7,"Energetic,Happy,Talkative,Uplifted","Pine,Diesel",3rd Coast Panama Chunk by Bedford Grow is a cr...
4,11,501St-Og,hybrid,4.5,"Relaxed,Euphoric,Sleepy,Happy,Talkative","Earthy,Pine,Skunk","501st OG, bred by Rare Dankness, is an indica-..."


,Strain,Type,Rating,Description,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5
0,98-White-Widow,hybrid,4.7,The ‘98 Aloha White Widow is an especially pot...,Relaxed,Aroused,Creative,Happy,Energetic
1,24K-Gold,hybrid,4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy,Relaxed,Euphoric,Uplifted,Talkative
2,3D-Cbd,sativa,4.6,3D CBD from Snoop Dogg’s branded line of canna...,Uplifted,Focused,Happy,Talkative,Relaxed
3,3Rd-Coast-Panama-Chunk,hybrid,4.7,3rd Coast Panama Chunk by Bedford Grow is a cr...,Energetic,Happy,Talkative,Uplifted,None
4,501St-Og,hybrid,4.5,"501st OG, bred by Rare Dankness, is an indica-...",Relaxed,Euphoric,Sleepy,Happy,Talkative


In [4]:
dummy = pd.get_dummies(data[['Effect_1','Effect_2','Effect_3','Effect_4','Effect_5']])
data = pd.concat([data, dummy], axis=1)

In [5]:
data = data.drop(['Effect_1','Effect_2','Effect_3','Effect_4',
                        'Effect_5',"Description"], axis=1)
data.head()

,Strain,Type,Rating,Effect_1_Aroused,Effect_1_Creative,Effect_1_Energetic,Effect_1_Euphoric,Effect_1_Focused,Effect_1_Giggly,Effect_1_Happy,Effect_1_Hungry,Effect_1_None,Effect_1_Relaxed,Effect_1_Sleepy,Effect_1_Talkative,Effect_1_Tingly,Effect_1_Uplifted,Effect_2_Aroused,Effect_2_Creative,Effect_2_Energetic,Effect_2_Euphoric,Effect_2_Focused,Effect_2_Giggly,Effect_2_Happy,Effect_2_Hungry,Effect_2_Relaxed,Effect_2_Sleepy,Effect_2_Talkative,Effect_2_Tingly,Effect_2_Uplifted,Effect_3_Aroused,Effect_3_Creative,Effect_3_Energetic,Effect_3_Euphoric,Effect_3_Focused,Effect_3_Giggly,Effect_3_Happy,Effect_3_Hungry,Effect_3_Relaxed,Effect_3_Sleepy,Effect_3_Talkative,Effect_3_Tingly,Effect_3_Uplifted,Effect_4_Aroused,Effect_4_Creative,Effect_4_Energetic,Effect_4_Euphoric,Effect_4_Focused,Effect_4_Giggly,Effect_4_Happy,Effect_4_Hungry,Effect_4_Relaxed,Effect_4_Sleepy,Effect_4_Talkative,Effect_4_Tingly,Effect_4_Uplifted,Effect_5_Aroused,Effect_5_Creative,Effect_5_Energetic,Effect_5_Euphoric,Effect_5_Focused,Effect_5_Giggly,Effect_5_Happy,Effect_5_Hungry,Effect_5_Relaxed,Effect_5_Sleepy,Effect_5_Talkative,Effect_5_Tingly,Effect_5_Uplifted
0,98-White-Widow,hybrid,4.7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,24K-Gold,hybrid,4.6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3D-Cbd,sativa,4.6,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3Rd-Coast-Panama-Chunk,hybrid,4.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,501St-Og,hybrid,4.5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
del data["Type"]
del data["Rating"]

In [7]:
d1 =data.set_index('Strain')

In [8]:
d1.T.head()
d1.shape

Strain,98-White-Widow,24K-Gold,3D-Cbd,3Rd-Coast-Panama-Chunk,501St-Og,818-Og,831-Og,9-Pound-Hammer,91-Krypt,Acdc,Abusive-Og,Acapulco-Gold,Ace-Killer-Og,Acid-Dough,Afghan-Sour-Kush,Afghan-Widow,Afghani-Cbd,Afghanimal,Afternoon-Delight,Agent-Tangie,Air-Jordan-Og,Alaska,Alaska-Thunder-Grape,Albert-Walker,Alchemy,Alf,Alice-In-Wonderland,Alien-Asshat,Alien-Bubba,Alien-Inferno,Alien-Og,Alien-Reunion,Alien-Rift,Alien-Rock-Candy,Alien-Sour-Apple,Alien-Technology,Aliens-On-Moonshine,Allen-Iverson-Og,Allkush,Aloha-Grape-Stomper,Aloha-Limone,Alpha-Blue,Alpha-Cow,Alpha-Express,Alpha-Og,Alpine-Blue,American-Dream,American-Kush,Americano,Amnesia-Ganja-Haze,Ancient-Kush,Ancient-Og,Angel-Og,Animal-Cookies,Anonymous-Og,Anubis,Ape-Shit,Appalachia,Apple-Kush,Appleberry,Arcticblue,Arjans-Ultra-Haze-1,Ash,Astroboy,Athabasca,Atmosphere,Atomic,Atomic-Goat,Avi-Dekel,B-Witched,B-Witched,B4,Bc-Big-Bud,Bc-Roadkill,Bcn-Diesel,Bsc,Bad-Azz-Kush,Banana-Cream-Og,Banana-Peel,Bananas,Bango,Batgirl,Bay-11,Bear-Dance,Beast-Mode-20,Bedford-Glue,Bediol,Bell-Ringer,Berry-Larry,Berts-Cookie-Dough,Bertberry-Cheesecake,Big-Band,Big-Kush,Big-Mac,Big-Sky-Og,Big-Smooth,Biker-Leblanc,Bio-Jesus,Birds-Eye,Birthday-Cake-Kush,Black-84,Black-Afghan,Black-Betty,Black-Bubba,Black-Cherry-Cheesecake,Black-Cherry-Pie,Black-Lime-Special-Reserve,Black-Mamba,Black-Tuna,Blackberry-Bubble,Blackberry-Cream,Blackberry-Fire,Blackberry-Haze,Blackberry-Lime-Haze,Blackberry-Pie,Blackwater,Blissful-Wizard,Blob-Og,Bloos-Kloos,Blood-Orange-Kush,Blue-Alien,Blue-Buddha,Blue-Cinex,Blue-Cookies,Blue-Dot,Blue-Frost,Blue-Galaxy,Blue-God,Blue-Heron,Blue-Jay-Way,Blue-Kiss,Blue-Knight,Blue-Kush,Blue-Lights,Blue-Magic,Blue-Magoo,Blue-Monster,Blue-Moon-Rocks,Blue-Nightmare,Blue-Nina,Blue-Ox,Blue-Power,Blue-Sky,Blue-Treat,Blue-Venom,Blue-Wonder,Blue-Wreck,Blue-Zkittlez,Blue-Zombie,Blueberry-Ak,Blueberry-Bomb,Blueberry-Dream,Blueberry-Essence,Blueberry-Lambsbread,Blueberry-Muffins,Blueberry-Og,Blueberry-Pancakes,Blueberry-Silvertip,Blueberry-Space-Cake,Blueberry-Waltz,Blueberry-Yum-Yum,Blukashima,Bluniverse,Blush,Bob-Saget,Booger,Bordello,Boss-Hogg,Brain-Candy,Brainwreck,Brand-X,Brandywine,Brazil-Amazonia,Brian-Berry-Cough,Bright-Fire,Broke-Diesel,Brooklyn-Mango,Bruce-Banner,Bruce-Banner-3,Bubba-Berry,Bubba-Fresh,Bubba-Tom-Hayes,Bubba-X-Skunk,Bubbas-Gift,Buddha-Haze,Buddha-Tahoe,Buddhas-Hand,Bugatti-Og,Burkle,C3,C4,Cbd-Critical-Cure,Cbd-Critical-Mass,Cbd-Kush,Cbd-Lilly,Cbd-Ox,Cbd-Shark,Cactus,Cactus-Cooler,Cali-Gold,Cali-Jack,California-Hash-Plant,Cambodian-Haze,Candyland,Canna-Tsu,Cannasutra,Cannalope-Kush,Capers,Captain-America-Og,Captains-Cake,Caramel-Candy-Kush,Caramel-Kona-Coffee-Kush,Caramella,Cascadia-Kush,Casper-Og,Catfish,Charlie-Sheen,Charlottes-Web,Chem-Dog,Chem-Jack,Chem-Jong-Ill,Chem-Scout,Chems-Sister,Chemwreck,Chemdawg-4-X-Alien-Og-Kush,Chemdawg-91,Chemdawg-Sour-Diesel,Chemmy-Jones,Chemnesia,Cherwillie,Cherry-Cookies,Cherry-Diesel,Cherry-Kola,Cherry-Sauce,Cherry-Slyder,Cherry-Thunder-Fuck,Chicle,Chiquita-Banana,Chloe,Chocolate-Diesel,Chocolate-Drop,Chocolate-Fondue,Chocolate-Hashberry,Chocolate-Kush,Chocolate-Og,Chocolate-Rain,Chocolate-Sap,Chong-Star,Chronic-Thunder,Chuckys-Bride,Chunky-Diesel,Chupacabra,Cindy-White,Cirrus,Citrus-Sunshine-Haze,Clementines-Terpentine,Clockwork-Orange,Club-69,Colombian-Gold,Colombian-Mojito,Colorado-Bubba,Colorado-Clementines,Confidential-Cheese,Congo,Conspiracy-Kush,Cookie-Breath,Cookie-Dough,Cookie-Jar,Cookie-Monster,Cookie-Wreck,Cookies-Kush,Cookies-And-Cream,Cookies-And-Dream,Corleone-Kush,Cornbread,Cowboy-Kush,Crater-Lake,Crazy-Miss-Hyde,Critical-Kali-Mist,Critical-Kush,Critical-Skunk,Cronuts,Crouching-Tiger-Hidden-Alien,Crown-Og,Crown-Royale,Crystal-Coma,Crystalberry,Cuvee,Dj-Andy-Williams,Dakini-Kush,Dance-World,Dancehall,Dank-Schrader,Dank-Sinatra,Dark-Blue-Dream,Dark-Side-Of-The-Moon,Darkside-Og,Dawgs-Waltz,Death-Bubba,Death-Star,Deathstar-Og,Deep-Sleep,Delahaze,Desert-Star,Destroyer,Diamond-Og,Diamond-Valley-Kush,Dinachem,Dirty-Girl,Dirty-Old-Bastard,

(1151, 66)

In [9]:
d1.loc['prueba'] = 0
d1.loc['prueba'] = {'Effect_1_Aroused':1,'Effect_2_Sleepy':1}
d1 = d1.fillna(0)
d1.tail()
d2 = d1.T
d2.tail()

,Effect_1_Aroused,Effect_1_Creative,Effect_1_Energetic,Effect_1_Euphoric,Effect_1_Focused,Effect_1_Giggly,Effect_1_Happy,Effect_1_Hungry,Effect_1_None,Effect_1_Relaxed,Effect_1_Sleepy,Effect_1_Talkative,Effect_1_Tingly,Effect_1_Uplifted,Effect_2_Aroused,Effect_2_Creative,Effect_2_Energetic,Effect_2_Euphoric,Effect_2_Focused,Effect_2_Giggly,Effect_2_Happy,Effect_2_Hungry,Effect_2_Relaxed,Effect_2_Sleepy,Effect_2_Talkative,Effect_2_Tingly,Effect_2_Uplifted,Effect_3_Aroused,Effect_3_Creative,Effect_3_Energetic,Effect_3_Euphoric,Effect_3_Focused,Effect_3_Giggly,Effect_3_Happy,Effect_3_Hungry,Effect_3_Relaxed,Effect_3_Sleepy,Effect_3_Talkative,Effect_3_Tingly,Effect_3_Uplifted,Effect_4_Aroused,Effect_4_Creative,Effect_4_Energetic,Effect_4_Euphoric,Effect_4_Focused,Effect_4_Giggly,Effect_4_Happy,Effect_4_Hungry,Effect_4_Relaxed,Effect_4_Sleepy,Effect_4_Talkative,Effect_4_Tingly,Effect_4_Uplifted,Effect_5_Aroused,Effect_5_Creative,Effect_5_Energetic,Effect_5_Euphoric,Effect_5_Focused,Effect_5_Giggly,Effect_5_Happy,Effect_5_Hungry,Effect_5_Relaxed,Effect_5_Sleepy,Effect_5_Talkative,Effect_5_Tingly,Effect_5_Uplifted
Strain,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Zeus-Og,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Zombie-Kush,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Zoom-Pie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
prueba,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Strain,98-White-Widow,24K-Gold,3D-Cbd,3Rd-Coast-Panama-Chunk,501St-Og,818-Og,831-Og,9-Pound-Hammer,91-Krypt,Acdc,Abusive-Og,Acapulco-Gold,Ace-Killer-Og,Acid-Dough,Afghan-Sour-Kush,Afghan-Widow,Afghani-Cbd,Afghanimal,Afternoon-Delight,Agent-Tangie,Air-Jordan-Og,Alaska,Alaska-Thunder-Grape,Albert-Walker,Alchemy,Alf,Alice-In-Wonderland,Alien-Asshat,Alien-Bubba,Alien-Inferno,Alien-Og,Alien-Reunion,Alien-Rift,Alien-Rock-Candy,Alien-Sour-Apple,Alien-Technology,Aliens-On-Moonshine,Allen-Iverson-Og,Allkush,Aloha-Grape-Stomper,Aloha-Limone,Alpha-Blue,Alpha-Cow,Alpha-Express,Alpha-Og,Alpine-Blue,American-Dream,American-Kush,Americano,Amnesia-Ganja-Haze,Ancient-Kush,Ancient-Og,Angel-Og,Animal-Cookies,Anonymous-Og,Anubis,Ape-Shit,Appalachia,Apple-Kush,Appleberry,Arcticblue,Arjans-Ultra-Haze-1,Ash,Astroboy,Athabasca,Atmosphere,Atomic,Atomic-Goat,Avi-Dekel,B-Witched,B-Witched,B4,Bc-Big-Bud,Bc-Roadkill,Bcn-Diesel,Bsc,Bad-Azz-Kush,Banana-Cream-Og,Banana-Peel,Bananas,Bango,Batgirl,Bay-11,Bear-Dance,Beast-Mode-20,Bedford-Glue,Bediol,Bell-Ringer,Berry-Larry,Berts-Cookie-Dough,Bertberry-Cheesecake,Big-Band,Big-Kush,Big-Mac,Big-Sky-Og,Big-Smooth,Biker-Leblanc,Bio-Jesus,Birds-Eye,Birthday-Cake-Kush,Black-84,Black-Afghan,Black-Betty,Black-Bubba,Black-Cherry-Cheesecake,Black-Cherry-Pie,Black-Lime-Special-Reserve,Black-Mamba,Black-Tuna,Blackberry-Bubble,Blackberry-Cream,Blackberry-Fire,Blackberry-Haze,Blackberry-Lime-Haze,Blackberry-Pie,Blackwater,Blissful-Wizard,Blob-Og,Bloos-Kloos,Blood-Orange-Kush,Blue-Alien,Blue-Buddha,Blue-Cinex,Blue-Cookies,Blue-Dot,Blue-Frost,Blue-Galaxy,Blue-God,Blue-Heron,Blue-Jay-Way,Blue-Kiss,Blue-Knight,Blue-Kush,Blue-Lights,Blue-Magic,Blue-Magoo,Blue-Monster,Blue-Moon-Rocks,Blue-Nightmare,Blue-Nina,Blue-Ox,Blue-Power,Blue-Sky,Blue-Treat,Blue-Venom,Blue-Wonder,Blue-Wreck,Blue-Zkittlez,Blue-Zombie,Blueberry-Ak,Blueberry-Bomb,Blueberry-Dream,Blueberry-Essence,Blueberry-Lambsbread,Blueberry-Muffins,Blueberry-Og,Blueberry-Pancakes,Blueberry-Silvertip,Blueberry-Space-Cake,Blueberry-Waltz,Blueberry-Yum-Yum,Blukashima,Bluniverse,Blush,Bob-Saget,Booger,Bordello,Boss-Hogg,Brain-Candy,Brainwreck,Brand-X,Brandywine,Brazil-Amazonia,Brian-Berry-Cough,Bright-Fire,Broke-Diesel,Brooklyn-Mango,Bruce-Banner,Bruce-Banner-3,Bubba-Berry,Bubba-Fresh,Bubba-Tom-Hayes,Bubba-X-Skunk,Bubbas-Gift,Buddha-Haze,Buddha-Tahoe,Buddhas-Hand,Bugatti-Og,Burkle,C3,C4,Cbd-Critical-Cure,Cbd-Critical-Mass,Cbd-Kush,Cbd-Lilly,Cbd-Ox,Cbd-Shark,Cactus,Cactus-Cooler,Cali-Gold,Cali-Jack,California-Hash-Plant,Cambodian-Haze,Candyland,Canna-Tsu,Cannasutra,Cannalope-Kush,Capers,Captain-America-Og,Captains-Cake,Caramel-Candy-Kush,Caramel-Kona-Coffee-Kush,Caramella,Cascadia-Kush,Casper-Og,Catfish,Charlie-Sheen,Charlottes-Web,Chem-Dog,Chem-Jack,Chem-Jong-Ill,Chem-Scout,Chems-Sister,Chemwreck,Chemdawg-4-X-Alien-Og-Kush,Chemdawg-91,Chemdawg-Sour-Diesel,Chemmy-Jones,Chemnesia,Cherwillie,Cherry-Cookies,Cherry-Diesel,Cherry-Kola,Cherry-Sauce,Cherry-Slyder,Cherry-Thunder-Fuck,Chicle,Chiquita-Banana,Chloe,Chocolate-Diesel,Chocolate-Drop,Chocolate-Fondue,Chocolate-Hashberry,Chocolate-Kush,Chocolate-Og,Chocolate-Rain,Chocolate-Sap,Chong-Star,Chronic-Thunder,Chuckys-Bride,Chunky-Diesel,Chupacabra,Cindy-White,Cirrus,Citrus-Sunshine-Haze,Clementines-Terpentine,Clockwork-Orange,Club-69,Colombian-Gold,Colombian-Mojito,Colorado-Bubba,Colorado-Clementines,Confidential-Cheese,Congo,Conspiracy-Kush,Cookie-Breath,Cookie-Dough,Cookie-Jar,Cookie-Monster,Cookie-Wreck,Cookies-Kush,Cookies-And-Cream,Cookies-And-Dream,Corleone-Kush,Cornbread,Cowboy-Kush,Crater-Lake,Crazy-Miss-Hyde,Critical-Kali-Mist,Critical-Kush,Critical-Skunk,Cronuts,Crouching-Tiger-Hidden-Alien,Crown-Og,Crown-Royale,Crystal-Coma,Crystalberry,Cuvee,Dj-Andy-Williams,Dakini-Kush,Dance-World,Dancehall,Dank-Schrader,Dank-Sinatra,Dark-Blue-Dream,Dark-Side-Of-The-Moon,Darkside-Og,Dawgs-Waltz,Death-Bubba,Death-Star,Deathstar-Og,Deep-Sleep,Delahaze,Desert-Star,Destroyer,Diamond-Og,Diamond-Valley-Kush,Dinachem,Dirty-Girl,Dirty-Old-Bastard,

In [10]:

from scipy.spatial.distance import pdist, squareform

squareform(pdist(d2.T, 'cosine'))

array([[0.        , 1.        , 1.        , ..., 0.8       , 0.8       ,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 0.8       ,
        1.        ],
       [1.        , 1.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.8       , 1.        , 1.        , ..., 0.        , 1.        ,
        0.68377223],
       [0.8       , 0.8       , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.68377223, 1.        ,
        0.        ]])

In [11]:
from scipy.spatial.distance import pdist, squareform
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

distances = pd.DataFrame(1/(1 + squareform(pdist(d2.T, 'cosine'))), 
                         index=d2.columns, columns=d2.columns)

distances.head()

Strain,98-White-Widow,24K-Gold,3D-Cbd,3Rd-Coast-Panama-Chunk,501St-Og,818-Og,831-Og,9-Pound-Hammer,91-Krypt,Acdc,Abusive-Og,Acapulco-Gold,Ace-Killer-Og,Acid-Dough,Afghan-Sour-Kush,Afghan-Widow,Afghani-Cbd,Afghanimal,Afternoon-Delight,Agent-Tangie,Air-Jordan-Og,Alaska,Alaska-Thunder-Grape,Albert-Walker,Alchemy,Alf,Alice-In-Wonderland,Alien-Asshat,Alien-Bubba,Alien-Inferno,Alien-Og,Alien-Reunion,Alien-Rift,Alien-Rock-Candy,Alien-Sour-Apple,Alien-Technology,Aliens-On-Moonshine,Allen-Iverson-Og,Allkush,Aloha-Grape-Stomper,Aloha-Limone,Alpha-Blue,Alpha-Cow,Alpha-Express,Alpha-Og,Alpine-Blue,American-Dream,American-Kush,Americano,Amnesia-Ganja-Haze,Ancient-Kush,Ancient-Og,Angel-Og,Animal-Cookies,Anonymous-Og,Anubis,Ape-Shit,Appalachia,Apple-Kush,Appleberry,Arcticblue,Arjans-Ultra-Haze-1,Ash,Astroboy,Athabasca,Atmosphere,Atomic,Atomic-Goat,Avi-Dekel,B-Witched,B-Witched,B4,Bc-Big-Bud,Bc-Roadkill,Bcn-Diesel,Bsc,Bad-Azz-Kush,Banana-Cream-Og,Banana-Peel,Bananas,Bango,Batgirl,Bay-11,Bear-Dance,Beast-Mode-20,Bedford-Glue,Bediol,Bell-Ringer,Berry-Larry,Berts-Cookie-Dough,Bertberry-Cheesecake,Big-Band,Big-Kush,Big-Mac,Big-Sky-Og,Big-Smooth,Biker-Leblanc,Bio-Jesus,Birds-Eye,Birthday-Cake-Kush,Black-84,Black-Afghan,Black-Betty,Black-Bubba,Black-Cherry-Cheesecake,Black-Cherry-Pie,Black-Lime-Special-Reserve,Black-Mamba,Black-Tuna,Blackberry-Bubble,Blackberry-Cream,Blackberry-Fire,Blackberry-Haze,Blackberry-Lime-Haze,Blackberry-Pie,Blackwater,Blissful-Wizard,Blob-Og,Bloos-Kloos,Blood-Orange-Kush,Blue-Alien,Blue-Buddha,Blue-Cinex,Blue-Cookies,Blue-Dot,Blue-Frost,Blue-Galaxy,Blue-God,Blue-Heron,Blue-Jay-Way,Blue-Kiss,Blue-Knight,Blue-Kush,Blue-Lights,Blue-Magic,Blue-Magoo,Blue-Monster,Blue-Moon-Rocks,Blue-Nightmare,Blue-Nina,Blue-Ox,Blue-Power,Blue-Sky,Blue-Treat,Blue-Venom,Blue-Wonder,Blue-Wreck,Blue-Zkittlez,Blue-Zombie,Blueberry-Ak,Blueberry-Bomb,Blueberry-Dream,Blueberry-Essence,Blueberry-Lambsbread,Blueberry-Muffins,Blueberry-Og,Blueberry-Pancakes,Blueberry-Silvertip,Blueberry-Space-Cake,Blueberry-Waltz,Blueberry-Yum-Yum,Blukashima,Bluniverse,Blush,Bob-Saget,Booger,Bordello,Boss-Hogg,Brain-Candy,Brainwreck,Brand-X,Brandywine,Brazil-Amazonia,Brian-Berry-Cough,Bright-Fire,Broke-Diesel,Brooklyn-Mango,Bruce-Banner,Bruce-Banner-3,Bubba-Berry,Bubba-Fresh,Bubba-Tom-Hayes,Bubba-X-Skunk,Bubbas-Gift,Buddha-Haze,Buddha-Tahoe,Buddhas-Hand,Bugatti-Og,Burkle,C3,C4,Cbd-Critical-Cure,Cbd-Critical-Mass,Cbd-Kush,Cbd-Lilly,Cbd-Ox,Cbd-Shark,Cactus,Cactus-Cooler,Cali-Gold,Cali-Jack,California-Hash-Plant,Cambodian-Haze,Candyland,Canna-Tsu,Cannasutra,Cannalope-Kush,Capers,Captain-America-Og,Captains-Cake,Caramel-Candy-Kush,Caramel-Kona-Coffee-Kush,Caramella,Cascadia-Kush,Casper-Og,Catfish,Charlie-Sheen,Charlottes-Web,Chem-Dog,Chem-Jack,Chem-Jong-Ill,Chem-Scout,Chems-Sister,Chemwreck,Chemdawg-4-X-Alien-Og-Kush,Chemdawg-91,Chemdawg-Sour-Diesel,Chemmy-Jones,Chemnesia,Cherwillie,Cherry-Cookies,Cherry-Diesel,Cherry-Kola,Cherry-Sauce,Cherry-Slyder,Cherry-Thunder-Fuck,Chicle,Chiquita-Banana,Chloe,Chocolate-Diesel,Chocolate-Drop,Chocolate-Fondue,Chocolate-Hashberry,Chocolate-Kush,Chocolate-Og,Chocolate-Rain,Chocolate-Sap,Chong-Star,Chronic-Thunder,Chuckys-Bride,Chunky-Diesel,Chupacabra,Cindy-White,Cirrus,Citrus-Sunshine-Haze,Clementines-Terpentine,Clockwork-Orange,Club-69,Colombian-Gold,Colombian-Mojito,Colorado-Bubba,Colorado-Clementines,Confidential-Cheese,Congo,Conspiracy-Kush,Cookie-Breath,Cookie-Dough,Cookie-Jar,Cookie-Monster,Cookie-Wreck,Cookies-Kush,Cookies-And-Cream,Cookies-And-Dream,Corleone-Kush,Cornbread,Cowboy-Kush,Crater-Lake,Crazy-Miss-Hyde,Critical-Kali-Mist,Critical-Kush,Critical-Skunk,Cronuts,Crouching-Tiger-Hidden-Alien,Crown-Og,Crown-Royale,Crystal-Coma,Crystalberry,Cuvee,Dj-Andy-Williams,Dakini-Kush,Dance-World,Dancehall,Dank-Schrader,Dank-Sinatra,Dark-Blue-Dream,Dark-Side-Of-The-Moon,Darkside-Og,Dawgs-Waltz,Death-Bubba,Death-Star,Deathstar-Og,Deep-Sleep,Delahaze,Desert-Star,Destroyer,Diamond-Og,Diamond-Valley-Kush,Dinachem,Dirty-Girl,Dirty-Old-Bastard,

In [12]:
top5 = distances['prueba'].sort_values(ascending=False)[1:6]
top5

Strain
Jack-Burton            0.731238
Cbd-Ox                 0.731238
Truffle-Butter         0.731238
Bubba-Fresh            0.731238
White-Tahoe-Cookies    0.731238
Name: prueba, dtype: float64

In [14]:
prueba5= cannabis[cannabis["Strain"] == 'Jack-Burton']
prueba5

,Strain,Type,Rating,Effects,Flavor,Description
1095,Jack-Burton,indica,4.5,"Aroused,Sleepy,Uplifted,Creative,Euphoric","Sweet,Flowery,Rose",Jack Burton by THClones is a sensuous indica-d...
